### Text generation using `Fnet`.

In this notebook we are going to create a model that does text generation by following [keras examples tutorial](https://keras.io/examples/nlp/text_generation_fnet/)
based on the [FNet](https://arxiv.org/abs/2105.03824) achitecture.

> The original transformer implementation (Vaswani et al., 2017) was one of the major breakthroughs in Natural Language Processing, giving rise to important architectures such BERT and GPT. However, the drawback of these architectures is that the self-attention mechanism they use is computationally expensive. The FNet architecture proposes to replace this self-attention attention with a leaner mechanism: a Fourier transformation-based linear mixer for input tokens.


> The FNet model was able to achieve ``92-97%`` of BERT's accuracy while training ``80%`` faster on GPUs and almost ``70%`` faster on TPUs. This type of design provides an efficient and small model size, leading to faster inference times.

In this notebook we will implement text generation using [Cornell_Movie-Dialogs_Corpus dataset.](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)


### Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
import re, os

In [2]:
tf.__version__

'2.6.0'

### Hyper parameters

In [3]:
VOCAB_SIZE = 8192
MAX_SAMPLES = 50000
BUFFER_SIZE = 20000
MAX_LENGTH = 40
EMBED_DIM = 256
LATENT_DIM = 512
NUM_HEADS = 8
BATCH_SIZE = 64

### Loading the data.

The dataset that we will be working with is the [Cornell Dialog Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html) which contains converstions inform of questions and answers set.

In [ ]:
path_to_zip = keras.utils.get_file(
    "cornell_movie_dialogs.zip",
    origin="http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip",
    extract=True,
)

path_to_dataset = os.path.join(
    os.path.dirname(path_to_zip), "cornell movie-dialogs corpus"
)
path_to_movie_lines = os.path.join(path_to_dataset, "movie_lines.txt")
path_to_movie_conversations = os.path.join(path_to_dataset, "movie_conversations.txt")


In [14]:
def load_conversations():
  id2line = {}
  with open(path_to_movie_lines, errors="ignore") as file:
    lines = file.readlines()
  for line in lines:
    parts  = line.replace("\n", "").split(" +++$+++ ")
    id2line[parts[0]] = parts[4]

  inputs, outputs = [], []
  with open(path_to_movie_conversations, "r") as file:
    lines = file.readlines()
  
  for line in lines:
    parts = line.replace("\n", "").split(" +++$+++ ")
    conversation = [line[1:-1] for line in parts[3][1:-1].split(", ")]
    
    for i in range(len(conversation) - 1):
      inputs.append(id2line[conversation[i]])
      outputs.append(id2line[conversation[i + 1]])
      if len(inputs) >= MAX_SAMPLES:
          return inputs, outputs
  return inputs, outputs

In [15]:
questions, answers = load_conversations()

### Splitting the data into sets

We are then going to split our dataset into two sets:
1. train
2. validation

In [22]:
train_dataset = tf.data.Dataset.from_tensor_slices((questions[:40000], answers[:40000]))
val_dataset = tf.data.Dataset.from_tensor_slices((questions[40000:], answers[40000:]))

### Checking 10 questions and answers pairs.

In [17]:
for i in range(100, 110):
  print("> question: ", questions[i])
  print("> answer: ", answers[i])
  print("*"*50)

> question:  You set me up.
> answer:  I just wanted --
**************************************************
> question:  I just wanted --
> answer:  What? To completely damage me?  To send me to therapy forever? What?
**************************************************
> question:  What? To completely damage me?  To send me to therapy forever? What?
> answer:  No! I just wanted
**************************************************
> question:  Is that woman a complete fruit-loop or is it just me?
> answer:  It's just you.
**************************************************
> question:  Patrick -- is that- a.
> answer:  Perm?
**************************************************
> question:  Now don't get upset. Daddy, but there's this boy... and I think he might ask...
> answer:  No! You're not dating until your sister starts dating.  End of discussion.
**************************************************
> question:  No! You're not dating until your sister starts dating.  End of discussion.
> an

### Processing the text

We are going to use `tf.strings` to clean our text. The function `process_text` does that for us.

In [18]:
def process_text(sentence):
  sentence = tf.strings.lower(sentence)
  sentence = tf.strings.regex_replace(sentence, r"([?.!,])", r" \1 ")
  sentence = tf.strings.regex_replace(sentence, r"\s\s+", " ")
  entence = tf.strings.regex_replace(sentence, r"[^a-z?.!,]+", " ")
  sentence = tf.strings.strip(sentence)
  sentence = tf.strings.join(["<start>", sentence, "<end>"], separator=" ")
  return sentence

### Text vectorixation layer.

We are going to then create a `TextVectorization` layer that will handle text vectorization for us. The text vectorization will adapt from our questions and answers coupus.

In [20]:
vectorizer = keras.layers.TextVectorization(
    VOCAB_SIZE,
    standardize=process_text,
    output_mode="int",
    output_sequence_length=MAX_LENGTH,
)

In [21]:
corpus = questions + answers
vectorizer.adapt(
    tf.data.Dataset.from_tensor_slices((corpus)).batch(128)
)

### Tokenizing and padding `TextVectorization`

In [23]:
def vectorize_text(inputs, outputs):
  inputs, outputs = vectorizer(inputs), vectorizer(outputs)
  outputs = tf.pad(outputs, [[0, 1]])
  return (
        {"encoder_inputs": inputs, "decoder_inputs": outputs[:-1]},
        {"outputs": outputs[1:]},
    )


In [24]:
train_dataset = train_dataset.map(vectorize_text, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(vectorize_text, num_parallel_calls=tf.data.AUTOTUNE)

train_dataset = (
    train_dataset.cache()
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
val_dataset = val_dataset.cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

### `FNet` encoder

> The ``FNet`` paper proposes a replacement for the standard attention mechanism used by the Transformer architecture (Vaswani et al., 2017).

![img](https://i.imgur.com/rLg47qU.png)


The outputs of the FFT layer are complex numbers. To avoid dealing with complex layers, only the real part (the magnitude) is extracted.

The dense layers that follow the Fourier transformation act as convolutions applied on the frequency domain.

In [33]:
class FNetEncoder(keras.layers.Layer):
  def __init__(self, embed_dim, dense_dim, **kwargs):
    super(FNetEncoder, self).__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim

    self.dense_proj = keras.Sequential(
            [
                keras.layers.Dense(dense_dim, activation="relu"),
                keras.layers.Dense(embed_dim),
            ]
    )
    self.layernorm_1 = keras.layers.LayerNormalization()
    self.layernorm_2 = keras.layers.LayerNormalization()

  def call(self, inputs):
    inp_complex = tf.cast(inputs, tf.complex64) # Casting the inputs to complex64
    # Projecting the inputs to the frequency domain using FFT2D and
    # extracting the real part of the output
    fft = tf.math.real(tf.signal.fft2d(inp_complex))
    proj_input = self.layernorm_1(inputs + fft)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input + proj_output)


### Creating a decoder

The decoder architecture remains the same as the one proposed by (Vaswani et al., 2017) in the original transformer architecture, consisting of an embedding, positional encoding, two masked multihead attention layers and finally the dense output layers. The architecture that follows is taken from Deep Learning with Python, second edition, chapter 11.

### Positional embedding layer

In [27]:
class PositionalEmbedding(keras.layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = keras.layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = keras.layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

### FNet decoder layer

In [28]:
class FNetDecoder(keras.layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(FNetDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                keras.layers.Dense(latent_dim, activation="relu"),
                keras.layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = keras.layers.LayerNormalization()
        self.layernorm_2 = keras.layers.LayerNormalization()
        self.layernorm_3 = keras.layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)


### Creating the model

we are going then to create a function `create_model()` that will create an fnet model for us.

In [29]:
def create_model():
  encoder_inputs = keras.layers.Input(shape=(None,), dtype="int32", name="encoder_inputs")
  x = PositionalEmbedding(MAX_LENGTH, VOCAB_SIZE, EMBED_DIM)(encoder_inputs)
  encoder_outputs = FNetEncoder(EMBED_DIM, LATENT_DIM)(x)
  encoder = keras.Model(encoder_inputs, encoder_outputs)
  decoder_inputs = keras.layers.Input(shape=(None,), dtype="int32", name="decoder_inputs")
  encoded_seq_inputs = keras.Input(
      shape=(None, EMBED_DIM), name="decoder_state_inputs"
  )
  x = PositionalEmbedding(MAX_LENGTH, VOCAB_SIZE, EMBED_DIM)(decoder_inputs)
  x = FNetDecoder(EMBED_DIM, LATENT_DIM, NUM_HEADS)(x, encoded_seq_inputs)
  x = keras.layers.Dropout(0.5)(x)
  decoder_outputs = keras.layers.Dense(VOCAB_SIZE, activation="softmax")(x)
  decoder = keras.Model(
      [decoder_inputs, encoded_seq_inputs], decoder_outputs, name="outputs"
  )
  decoder_outputs = decoder([decoder_inputs, encoder_outputs])
  fnet = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs, name="fnet")
  return fnet
  

### Model instance and compilation

In [34]:
fnet = create_model()
fnet.compile("adam", 
             loss="sparse_categorical_crossentropy", 
             metrics=["accuracy"])

### Training

We are going to train this model for a single epoch since it is for demostration.

In [36]:
fnet.fit(train_dataset, epochs=1, validation_data=val_dataset)

625/625 [==============================] - 2494s 4s/step - loss: 1.6285 - accuracy: 0.2648 - val_loss: 1.4496 - val_accuracy: 0.3017


### Model inference (making preditions).


In [37]:
VOCAB = vectorizer.get_vocabulary()

In [40]:
def decode_sentence(input_sentence):
    # Mapping the input sentence to tokens and adding start and end tokens
    tokenized_input_sentence = vectorizer(
        tf.constant("<start> " + process_text(input_sentence) + " <end>")
    )
    # Initializing the initial sentence consisting of only the start token.
    tokenized_target_sentence = tf.expand_dims(VOCAB.index("<start>"), 0)
    decoded_sentence = ""
    for i in range(MAX_LENGTH):
        # Get the predictions
        predictions = fnet.predict(
            {
                "encoder_inputs": tf.expand_dims(tokenized_input_sentence, 0),
                "decoder_inputs": tf.expand_dims(
                    tf.pad(
                        tokenized_target_sentence,
                        [[0, MAX_LENGTH - tf.shape(tokenized_target_sentence)[0]]],
                    ),
                    0,
                ),
            }
        )
        # Calculating the token with maximum probability and getting the corresponding word
        sampled_token_index = tf.argmax(predictions[0, i, :])
        sampled_token = VOCAB[sampled_token_index.numpy()]
        # If sampled token is the end token then stop generating and return the sentence
        if tf.equal(sampled_token_index, VOCAB.index("<end>")):
            break
        decoded_sentence += sampled_token + " "
        tokenized_target_sentence = tf.concat(
            [tokenized_target_sentence, [sampled_token_index]], 0
        )

    return decoded_sentence


In [41]:
decode_sentence("Where have you been all this time?")

"i don't know . "

In [42]:
decode_sentence("What is your name?")

"i don't know . "

### Ref

* [keras examples](https://keras.io/examples/nlp/text_generation_fnet/)
* [Fnet Paper](https://arxiv.org/abs/2105.03824)
* [Deep Learning with Python, second edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras)